## Подготовка

In [1]:
# Библиотеки
import csv
from collections import defaultdict

# Для форматирования вывода
BOLD = '\033[1m'
END = '\033[0m'

Считываем файл с данными, узнаем из него список синсетов и близких слов.

In [2]:
# Извлечение слова из строки вида 'word#X:Y'
def lexeme(str):
    """Возвращает word, если вид word#X
    Возвращает word, freq, если вид word#X:freq
    """
    if '#' in str:
        word, tail = str.split('#', 1)
    else:
        word, tail = str, None

    if tail:
        if ':' in tail:
            labels, tail = tail.split(':', 1)
        else:
            labels, tail = tail, None

    if tail:
        freq = float(tail)
        return word, freq
    else:
        #freq = 1
        return word

In [3]:
%%time
synsets = {}  # Словарь {id -> список синсетов}
index = defaultdict(list)  # Словарь {слово -> номера синсетов с упоминаниями}
lexicon = set()  # Набор всех слов в базе
relations = {}  # Словарь {id -> словарь близких слов}

# Считываем файл
with open('watset-mcl-mcl-joint-exp-linked.tsv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE)  

    # Перебираем строки и заполняем переменные synsets и relations словами.
    # Ключи - номер строки (с единицы).
    # Значения synsets - массивы строковых слов.
    # Значения relations - словари вида {слово -> частота}.
    for row in reader:
        synsets[int(row[0])] = [lexeme(word) for word in row[2].split(', ') if word]
        
        relations_dict = dict()
        for word in row[4].split(', '):
            if word:
                key, value = lexeme(word)
                relations_dict[key] = value
        relations[int(row[0])] = relations_dict

        # Закидываем номер строки в index для каждого слова.
        for word in synsets[int(row[0])]:
            index[word].append(int(row[0]))

        # Обновляем лексикон базы данных
        lexicon.update(synsets[int(row[0])])

Wall time: 895 ms


Создадим список предлогов, чтобы их выкинуть из будущего предложения.

In [4]:
prepositions = ['в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через',
                'с', 'у', 'и', 'за', 'над', 'для', 'об', 'под', 'про', 'то', 'это', 'а']

## Функции обработки

In [5]:
# Форматирование запроса в "рабочий" вид.
def formatting(sentence, word):
    """Возвращает массив слов, из которых состоит предложение за вычетом предлогов и запрашиваемого слова."""
    table = {ord(char): None for char in '!?,.;'}  # Удаляем посторонние символы
    sentence = sentence.translate(table)
    
    sentence = sentence.lower()  
    words_in_sentence = sentence.split(' ')  
    words_in_sentence.remove(word)  # Удаляем исходное слово из списка соседних по предложению слов.  
    
    words_in_sentence = [x for x in words_in_sentence if x not in prepositions] # Удаляем предлоги
    
    print(BOLD + "Соседние слова:" + END, words_in_sentence)
    return words_in_sentence

In [6]:
# Поиск в найденных строках слов, которые есть в искомом предложении.
def find_similarity(mentions_list, words_in_sentence):
    """Возвращает словарь, ключи - номера строк со словом, значения - баллы совпадения этой строки"""
    mentions_dict = dict()
    for i in mentions_list:
        mentions_dict[i] = 0
        for close_word in words_in_sentence:
            if (close_word in synsets[i]):
                mentions_dict[i] = mentions_dict[i] + 1
            if (close_word in relations[i]):
                mentions_dict[i] = mentions_dict[i] + float(relations[i][close_word])
    print(BOLD + 'Номера строк и количество совпадений' + END, mentions_dict)
    return mentions_dict

In [7]:
# Находим лучшую строку и процентное соотношение.
def find_best(mentions):
    summary = 0
    max = 0
    for i in mentions.keys():
        summary = summary + mentions[i]
        if (mentions[i] > max):
            max = mentions[i]
            index = i

    print(BOLD + 'Синонимы:' + END)
    if (summary > 0):
        print(synsets[index])
        print(BOLD + 'Связанные слова:' + END)
        for key in relations[index].keys():
            print(key)
        print(BOLD + 'Совпадение по базе данных:' + END, str(max/summary * 100) + "%")
    else:
        print("not found")

## Работа с входными данными

In [8]:
sentence = (
    'Если подросток добрый, уступчивый и хороший, '
    + 'то это часто воспринимается окружающими как проявление слабости его характера, '
    + 'как неспособность чётко высказать свою позицию.')
word = 'уступчивый'

print(sentence)

Если подросток добрый, уступчивый и хороший, то это часто воспринимается окружающими как проявление слабости его характера, как неспособность чётко высказать свою позицию.


In [9]:
words_in_sentence = formatting(sentence, word)  # Разбиваем предложение на набор слов
mentions_list = index[word]  # Находим синсеты с упоминанием слов
similarity = find_similarity(mentions_list, words_in_sentence)  # Считаем количество совпадений синсета и предложения
find_best(similarity)  # Находим лучший синсет

Соседние слова: ['если', 'подросток', 'добрый', 'хороший', 'часто', 'воспринимается', 'окружающими', 'как', 'проявление', 'слабости', 'его', 'характера', 'как', 'неспособность', 'чётко', 'высказать', 'свою', 'позицию']
Номера строк и количество совпадений {2: 1.551321, 1293: 0, 6299: 0}
Синонимы:
['уступчивый', 'мирный', 'тише_воды_ниже_травы', 'жалостливый', 'мешковатый', 'медлительный', 'добродушный', 'спокойный', 'милостливый', 'смиренный', 'схимник', 'доброжелательный', 'бесшумный', 'смиренник', 'благоволительный', 'добросердечный', 'безгневный', 'терпеливый', 'чуткий', 'милосердый', 'отзывчивый', 'смирный', 'гуманный', 'неразговорчивый', 'миролюбивый', 'безбурный', 'душевный', 'милостивый', 'добр', 'медленный', 'протяжный', 'человечный', 'тихий', 'безветренный', 'добродетельный', 'рахманный', 'невысказанный', 'добряк', 'благодушный', 'скромный', 'сострадательный', 'безмятежный', 'безжелчный', 'восприимчивый', 'беззвучный', 'благий', 'умолчательный', 'безропотный', 'божья_коровка',